[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/notebooks/water_intake_profile_calculator/)

# Select your IPTS 

<img src='__docs/__all/shift_enter.gif' />

In [ ]:
from __code import system
from __code.water_intake_profile_calculator import WaterIntakeProfileCalculator, WaterIntakeProfileSelector

from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_water_intake_profile.ui')
from __code.roi_selection_ui import Interface

system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Description 

This notebook will calculate the water intake profile vs time of a sample.

Here are the steps (**bold** for user input/manipulation)
 * **select the normalized images**
 * images sorted by time
 * **region of interest seleted**
 * profile of counts vs vertical-pixel calculated (**select integrated algorithm: mean/median/sum**)
 * water intake profile vs file index or vs time
 * **export profiles**

# Python Import 

In [ ]:
%gui qt

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets.widgets import interact
import numpy as np
import os
import ipywe.fileselector
from ipywidgets import widgets
from IPython.core.display import display, HTML
import pytz
import datetime

from NeuNorm.normalization import Normalization

from __code.metadata_handler import MetadataHandler
from __code import file_handler


# Select Images to Process

In [ ]:
o_water = WaterIntakeProfileCalculator(working_dir = system.System.get_working_dir())
o_water.select_data()

# Select Profile Region 

In [ ]:
o_gui = WaterIntakeProfileSelector(dict_data=o_water.dict_files)
o_gui.show()

# DEBUGGING FROM HERE 

In [ ]:
path = '/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-15177/Sample5_1min_files'

import glob
list_dsc = glob.glob(os.path.join(path,"*.dsc"))

import pandas as pd
import re
from __code.file_handler import read_ascii

In [ ]:
from __code.file_format_reader import DscReader
                
o_dsc_reader = DscReader(list_files=list_dsc)
o_dsc_reader.read()        
o_dsc_reader.build_coresponding_file_image_name()
o_dsc_reader.make_tif_file_name_the_key()        

In [ ]:
o_dsc_reader.dict_time_stamp_vs_tiff

In [ ]:
from collections import defaultdict

my_dict = defaultdict(lambda: {'time_stamp': 0, 'time_stamp_user_format': 'N/A'})
my_dict['a'] = {'time_stamp': 15, 'time_stamp_user_format': 10000}
my_dict['b']['time_stamp'] = 20
my_dict['b']['time_stamp_user_format'] = -20

In [ ]:
 _dict_time_stamp_vs_tiff = defaultdict(lambda: {'time_stamp': 0,
                                                        'time_stamp_user_format': 'N/A'})
_dict_metadata = o_dsc_reader.dict_metadata

for _key in _dict_metadata.keys():
    _tif_name = _dict_metadata[_key]['tif_file_name']
    _time_stamp = _dict_metadata[_key]['time_stamp']
    _time_stamp_user_format = _dict_metadata[_key]['time_stamp_user_format']
    _dict_time_stamp_vs_tiff[_tif_name]['time_stamp']= _time_stamp
    _dict_time_stamp_vs_tiff[_tif_name]['time_stamp_user_format'] = _time_stamp_user_format